## ToCupy

This notebook is used to showcase the ``ToCupy`` in a variety of scenarios, and validate the changes introduced in the stacking operators and solvers to work with mixed CPU/GPU scenarios (e.g. model on GPU and data on CPU) 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import scipy as sp

from scipy.linalg import block_diag
from pylops.utils                      import dottest
from pylops.utils.wavelets             import *
from pylops.utils.seismicevents        import *
from pylops.utils.tapers               import *
from pylops.basicoperators             import *

from pylops.optimization.basic import cg as pylops_cg
from pylops.optimization.basic import cgls as pylops_cgls
from pylops.optimization.basic import lsqr as pylops_lsqr

dtype = np.float32

## VStack

In [2]:
nops = 5 # number of operators
n, m = 4, 5

# Create operator
Ms = [np.random.normal(0, 1, (n, m)) for _ in range(nops)]
M = np.vstack(Ms)
Mop = MatrixMult(M, dtype=dtype)

# Create data
x = np.ones(m, dtype=dtype)
y = Mop @ x

# Invert
xinv = Mop / y
print(xinv)

[1. 1. 1. 1. 1.]


### VStack with numpy input and numpy output

In [3]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top = ToCupy(Mop.dims, dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop @ Top
    Mops.append(Mop)
Mops = VStack(Mops)

In [4]:
y = Mops * x.ravel()

xinv = pylops_cgls(Mops, y)[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y)[0]
print(xinv, type(y), type(xinv))

[1.        1.0000001 0.9999999 0.9999999 1.0000001] <class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1.         1.00000002 0.99999994 0.99999991 1.00000001] <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### VStack with cupy input and numpy output

In [5]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop
    Mops.append(Mop)
Mops = VStack(Mops, inoutengine=("numpy", "cupy"))

In [6]:
xcp = cp.asarray(x)
y = Mops * xcp.ravel()

xinv = pylops_cgls(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

[1.         0.99999994 0.99999994 0.99999994 1.        ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>
[1.         0.9999999  0.99999994 0.9999998  1.        ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>


## Blockdiag

In [7]:
nops = 5 # number of operators
n = 4

# Create operator
Ms = [np.diag((i + 1) * np.ones(n, dtype=dtype)) for i in range(nops)]
Ms = [M.T @ M for M in Ms]
M = block_diag(*Ms)
Mop = BlockDiag([MatrixMult(M, dtype=dtype) for M in Ms])

# Create data
x = np.ones(nops * n, dtype=dtype)
y = Mop @ x

# Invert
xinv = Mop / y
print(xinv)

[1.00000003 1.00000003 1.00000003 1.00000003 1.00000018 1.00000018
 1.00000018 1.00000018 0.99999999 0.99999999 0.99999999 0.99999999
 1.00000001 1.00000001 1.00000001 1.00000001 0.99999999 0.99999999
 0.99999999 0.99999999]


### Blockdiag with numpy input and numpy output

In [8]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop], dtype=dtype))
    Top = ToCupy(Mop.dims, dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop @ Top
    Mops.append(Mop)
Mops = BlockDiag(Mops, forceflat=True)

In [9]:
y = Mops * x.ravel()

xinv = pylops_cgls(Mops, y)[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y)[0]
print(xinv, type(y), type(xinv))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] <class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### Blockdiag with cupy input and numpy output

In [10]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop
    Mops.append(Mop)
Mops = BlockDiag(Mops, forceflat=True, inoutengine=("numpy", "cupy"))

In [11]:
xcp = cp.asarray(x)
y = Mops * xcp.ravel()

xinv = pylops_cg(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

xinv = pylops_cgls(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

[1.         1.         1.         1.         0.99999994 0.99999994
 0.99999994 0.99999994 1.0000001  1.0000001  1.0000001  1.0000001
 0.9999999  0.9999999  0.9999999  0.9999999  1.000001   1.000001
 1.000001   1.000001  ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>
[0.99999994 0.99999994 0.99999994 0.99999994 1.0000011  1.0000011
 1.0000011  1.0000011  0.99999654 0.99999654 0.99999654 0.99999654
 1.         1.         1.         1.         1.0000001  1.0000001
 1.0000001  1.0000001 ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>
[0.9999999  0.9999999  0.9999999  0.9999999  0.9999987  0.9999987
 0.9999987  0.9999987  0.9999997  0.9999997  0.9999997  0.9999997
 0.99999994 0.99999994 0.99999994 0.99999994 1.0000002  1.0000002
 1.0000002  1.0000002 ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>


## HStack

In [12]:
nops = 5 # number of operators
n, m = 4, 5

# Create operator
Ms = [np.random.normal(0, 1, (n, m)) for _ in range(nops)]
M = np.hstack(Ms)
Mop = MatrixMult(M, dtype=dtype)

# Create data
x = np.ones(m * nops, dtype=dtype)
y = Mop @ x

# Invert
xinv = pylops_lsqr(Mop, y, niter=10)[0]
print(xinv)

[-0.32982478  0.04155607 -0.10366513 -0.45825968 -0.33280285  0.6957457
  0.35038274  0.33068262  0.27927103  0.3954964  -0.07574363  0.88515874
  0.22037246  0.06106394  0.16029213 -0.07926578  0.48143328 -0.27046805
 -0.08611529  0.4688985  -0.2990618   0.0136094   0.64654577  0.07277334
  0.41753116]


### HStack with numpy input and numpy output

In [13]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top = ToCupy(Mop.dims, dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop @ Top
    Mops.append(Mop)
Mops = HStack(Mops)

In [14]:
y = Mops * x.ravel()

xinv = pylops_cgls(Mops, y)[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y)[0]
print(xinv, type(y), type(xinv))

[-0.3298248   0.04155609 -0.10366513 -0.45825967 -0.33280286  0.6957457
  0.35038278  0.33068264  0.27927104  0.3954964  -0.07574362  0.8851588
  0.22037247  0.06106395  0.16029215 -0.0792658   0.4814333  -0.2704681
 -0.0861153   0.4688985  -0.29906183  0.0136094   0.64654577  0.07277334
  0.41753116] <class 'numpy.ndarray'> <class 'numpy.ndarray'>
[-0.32982478  0.04155609 -0.10366513 -0.4582597  -0.33280286  0.69574577
  0.35038279  0.33068263  0.27927107  0.39549641 -0.07574363  0.88515879
  0.22037248  0.06106395  0.16029215 -0.07926579  0.48143332 -0.27046808
 -0.0861153   0.46889852 -0.29906185  0.0136094   0.64654579  0.07277333
  0.41753117] <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### HStack with cupy input and numpy output

In [15]:
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop
    Mops.append(Mop)
Mops = HStack(Mops, inoutengine=("numpy", "cupy"))

In [16]:
xcp = cp.asarray(x)
y = Mops * xcp.ravel()

xinv = pylops_cgls(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

xinv = pylops_lsqr(Mops, y, x0=cp.zeros_like(xcp))[0]
print(xinv, type(y), type(xinv))

[-0.3298248   0.04155609 -0.10366514 -0.4582597  -0.33280286  0.6957457
  0.35038278  0.33068264  0.27927104  0.3954964  -0.07574363  0.88515884
  0.22037247  0.06106395  0.16029215 -0.0792658   0.4814333  -0.2704681
 -0.08611531  0.46889853 -0.29906183  0.01360939  0.64654577  0.07277334
  0.41753116] <class 'numpy.ndarray'> <class 'cupy.ndarray'>
[-0.32982475  0.04155608 -0.10366513 -0.4582597  -0.3328029   0.69574577
  0.35038275  0.33068264  0.27927107  0.3954964  -0.07574362  0.88515884
  0.22037248  0.06106395  0.16029213 -0.07926579  0.4814333  -0.2704681
 -0.0861153   0.4688985  -0.29906183  0.01360941  0.64654577  0.07277334
  0.4175312 ] <class 'numpy.ndarray'> <class 'cupy.ndarray'>


In [12]:
nops, n, m = 3, 4, 5
Ms = [np.random.normal(0, 1, (n, m)) for _ in range(nops)]

# Create operator
Mops = []
for iop in range(nops):
    Mop = MatrixMult(cp.asarray(Ms[iop]), dtype=dtype)
    Top1 = ToCupy(Mop.dimsd, dtype=dtype)
    Mop = Top1.H @ Mop
    Mops.append(Mop)
Mops = VStack(Mops, inoutengine=("numpy", "cupy"))

# Create data and invert
x = cp.ones(m, dtype=dtype)
y = Mops @ x.ravel()
xest = pylops_cgls(Mops, y, x0=cp.zeros_like(x))[0]
xest

array([1.       , 1.       , 1.       , 1.       , 1.0000001],
      dtype=float32)